In [85]:
from google import genai
from google.genai import types

client = genai.Client()
messages = []

In [86]:
get_weather_function = {
        "name": "get_weather",
        "description": "해당 도시의 날씨를 조회하는 함수",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "날씨 정보를 얻으려는 도시 이름"
                }
            },
            "required": ["city"]
        }
    }

# Configure the client and tools
tools = types.Tool(function_declarations=[get_weather_function])
config = types.GenerateContentConfig(tools=[tools])

In [87]:
from google.genai import types

def get_weather(city):
    """해당 도시의 날씨를 조회하는 함수"""
    # 실제 구현은 여기에 추가 (예: API 호출)
    return f"{city}로 get_weather함수 호출했습니다!!"

# FunctionMap: 함수 이름과 실제 함수 객체를 매핑
FUNCTION_MAP = {
    "get_weather": get_weather,
    # 새로운 함수를 추가할 때마다 여기에 등록
}

def build_conversation(messages):
    """messages 리스트를 conversation 문자열로 변환"""
    conversation = ""
    for msg in messages:
        role = msg.get("role", "user")
        content = msg.get("content", "")
        function_call = msg.get("function_call")
        
        if role == "user":
            conversation += f"사용자: {content}\n\n"
        elif role == "assistant":
            if function_call:
                func_name = function_call.get("name", "")
                func_args = function_call.get("arguments", {})
                if content:
                    conversation += f"AI: {content}\n\n"
                conversation += f"AI가 {func_name} 함수를 호출하려고 합니다. 인자: {func_args}\n\n"
            else:
                conversation += f"AI: {content}\n\n"
        elif role == "function":
            function_name = msg.get("name", "")
            conversation += f"함수 {function_name} 실행 결과: {content}\n\n"
    
    return conversation

def should_use_tools(messages):
    """tools를 사용해야 하는지 판단 (마지막 메시지가 function이 아니면 True)"""
    if not messages:
        return True
    return messages[-1].get("role") != "function"

def extract_response_parts(response):
    """응답에서 텍스트와 함수 호출 정보 추출"""
    parts = response.candidates[0].content.parts
    text_parts = []
    function_call = None
    
    for part in parts:
        if hasattr(part, 'text') and part.text:
            text_parts.append(part.text)
        if hasattr(part, 'function_call') and part.function_call:
            function_call = part.function_call
    
    return "".join(text_parts), function_call

def execute_function(function_name, function_args):
    """함수를 실행하고 결과 반환"""
    if function_name not in FUNCTION_MAP:
        raise ValueError(f"알 수 없는 함수: {function_name}")
    
    function = FUNCTION_MAP[function_name]
    return function(**function_args)

def handle_function_call(function_call, text_parts):
    """함수 호출 처리"""
    function_name = function_call.name
    function_args = function_call.args
    
    print(f"Function to call: {function_name}")
    print(f"Arguments: {function_args}")
    
    # assistant의 함수 호출 요청을 messages에 저장
    assistant_content = {
        "role": "assistant",
        "content": "".join(text_parts) if text_parts else None,
        "function_call": {
            "name": function_name,
            "arguments": function_args
        }
    }
    messages.append(assistant_content)
    
    # 함수 실행
    try:
        result = execute_function(function_name, function_args)
        print(f"Function result: {result}")
        
        messages.append({
            "role": "function",
            "name": function_name,
            "content": str(result)
        })
    except Exception as e:
        error_msg = f"함수 실행 중 오류 발생: {str(e)}"
        print(f"Error: {error_msg}")
        messages.append({
            "role": "function",
            "name": function_name,
            "content": error_msg
        })
    
    # 함수 결과를 받아서 다시 모델에 전달
    call_ai()

def handle_text_response(response_text, last_msg_role):
    """텍스트 응답 처리"""
    if response_text:
        messages.append({"role": "assistant", "content": response_text})
        print(f"AI: {response_text}")
    elif last_msg_role == "function" and messages:
        # 응답이 비어있고 함수 결과를 받은 경우
        last_function_result = messages[-1].get("content", "")
        if last_function_result:
            response_text = f"함수 실행 결과: {last_function_result}"
            messages.append({"role": "assistant", "content": response_text})
            print(f"AI: {response_text}")
        else:
            print("AI: (응답 없음)")
    else:
        print("AI: (응답 없음)")

def call_ai():
    """AI 모델을 호출하고 응답 처리"""
    # 대화 히스토리를 conversation 문자열로 변환
    conversation = build_conversation(messages)
    
    # tools 사용 여부 결정
    use_tools = should_use_tools(messages)
    last_msg_role = messages[-1].get("role") if messages else None
    
    # 함수 결과를 받은 후에는 명확한 지시 추가
    if not use_tools:
        conversation += "위 함수 실행 결과를 바탕으로 사용자에게 자연스러운 텍스트로 응답해주세요. 더 이상 함수를 호출하지 마세요.\n\n"
    
    # API 호출
    try:
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=conversation,
            config=config if use_tools else None,
        )
    except Exception as e:
        print(f"API 호출 오류: {e}")
        return
    
    # 응답에서 텍스트와 함수 호출 정보 추출
    response_text, function_call = extract_response_parts(response)
    
    # 함수 호출이 있으면 처리, 없으면 텍스트 응답 처리
    if function_call:
        handle_function_call(function_call, [response_text] if response_text else [])
    else:
        handle_text_response(response_text, last_msg_role)

In [88]:
while True:
    ask = input("send a msg to the LLM: ")
    if ask == 'q' or ask == 'ㅂ':
        break   
    else:
        messages.append({"role": "user", "content": ask})
        print(f"User: {ask}")
        call_ai()
        print()  # 빈 줄 추가


User: 내 이름은 길재용
AI: 안녕하세요 길재용님, 무엇을 도와드릴까요?


User: 내 이름이 뭐라고?
AI: 길재용님이십니다.

User: 서울 날씨는?
Function to call: get_weather
Arguments: {'city': '서울'}
Function result: 서울로 get_weather함수 호출했습니다!!
AI: 서울 날씨 정보를 확인하기 위해 시스템을 호출했습니다. 현재로서는 정확한 서울 날씨 정보를 알려드리기는 어렵습니다.

User: 아이폰은 뭐야?
AI: 아이폰은 애플에서 디자인하고 마케팅하는 스마트폰 시리즈입니다.

제가 더 궁금한 점이 있으신가요?

User: 현재 아이폰은 몇 버전까지 나왔음?
AI: 죄송하지만 현재 아이폰이 몇 버전까지 나왔는지에 대한 정보는 가지고 있지 않습니다.

User: 그럼 내가 알고 있는 최신 버전은 몇년도고 무슨 버전인가?
AI: 죄송하지만, 제가 이전에 아이폰 버전에 대한 정보가 없다고 말씀드렸기 때문에, 고객님께서 알고 계신 최신 버전이 몇 년도이고 무슨 버전인지에 대한 정보도 제가 알려드릴 수 없습니다.

User: 구글은 뭐하는 회사임?
AI: 구글은 인터넷 검색, 클라우드 컴퓨팅, 소프트웨어, 하드웨어 등을 다루는 미국의 다국적 기술 기업입니다. 가장 잘 알려진 제품으로는 구글 검색 엔진, 안드로이드 운영체제, 유튜브, 크롬 웹 브라우저 등이 있습니다.

User: 좀 더 자세히 설명해주면 안될까?
AI: 구글은 전 세계적으로 가장 큰 영향력을 가진 기술 기업 중 하나입니다. 인터넷 검색 엔진으로 시작하여 현재는 인공지능, 자율주행차(웨이모), 생명 과학(칼리코) 등 다양한 분야로 사업을 확장했습니다.

주요 사업 분야를 좀 더 자세히 설명해 드릴게요:

1.  **인터넷 검색**: 구글의 핵심 사업이자 가장 잘 알려진 서비스입니다. 전 세계 웹사이트를 색인화하여 사용자들이 원하는 정보를 빠르게 찾을 수 있도록 돕습니다.
2.  **광고**: 구글의 주요 수익원 중 하나로,